# Stage 1: Generate Stock Universe

- Gather stocks of interest
- Gather stocks from specific criteria (SP500 top 50...)
- Gather stocks from specific portfolio account
- Assemble stock universe 
- Gather price histories

In [1]:
from platform import python_version
import time
from datetime import datetime
import os
import pandas as pd
import numpy as np
import math
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (20, 8)

# Set the import path for the tools directiory
import sys
# insert at position 1 in the path, as 0 is the path of this file.
sys.path.insert(1, '../tools')
import importlib
import ameritrade_functions as amc
importlib.reload(amc)
import utils
importlib.reload(utils)

print(f'Python version: {python_version()}')
print(f'Pandas version: {pd.__version__}')

Python version: 3.8.10
Pandas version: 0.25.3


## Configure Ameritrade Information

Ameritrade credentials are stored in environment variables to keep from having unencrypted passwords stored on disk.

The module automatically masks the account numbers to protect the actual accounts. An Ameritrade user can have many investment accounts. We will be working with only one for this demonstration.

## Authentication Tokens

To get data from Ameritrade you will need to obtains a short time use token (there is a re-use token, but I have not coded it yet.) You only need to do this if you
are going to use an existing Ameritrade account to define an initial set of stocks to analyze.

To obtain a token, you will need to have a Chrome driver located somewhere on your system. This will allow the module to use your credentials to obtain an authentication token.

For security reasons, I sugges using environment variables to store your credential information. If you store them in property files, or just code them into your notebook, you risk sharing the information with others if you use GitHub or some other SCCS. This also makes it easier to have them availabe from project to project in your development environment

<span style="color:blue">Note: *Account numbers are masked for security purposes.*</span>

In [23]:
username = os.getenv('maiotradeuser')
password = os.getenv('maiotradepw')
client_id = os.getenv('maiotradeclientid')

# For Chromedriver
from pathlib import Path
chrome_executabel_path = str(Path.home()) + r'\Anaconda Projects\chromedriver\chromedriver'

# Make sure we have a data directory
Path('./data').mkdir(parents=True, exist_ok=True) 

# Which account are we interested in
masked_account_number = '#---9216'
account_portfolios_file_name = 'data/portfolio_data.csv'
portfolio_file_name = 'data/portfolio_' + masked_account_number[-4:] + '.csv'
price_histories_file_name = 'data/price_histories.csv'

In [3]:
td_ameritrade = amc.AmeritradeRest(username, password, client_id, chrome_executabel_path)
td_ameritrade.authenticate()

if len(td_ameritrade.authorization) == 0:
    print('Error: No authorization data: {}'.format(td_ameritrade.authorization))
else:
    print('You have authorization')

You have authorization


## Stock Universe

Here we setup the univers. This needs some work. The long term goal is to use a pipeline process to help select stock that are in the top 500 or something similare.

For now we will use stocks from the portfolio, but stocks of interest (high news items), a list of well known stocks (this also has been augmented with some stocks that made Ameritrade's top 10 movers for a couple of days. This Ameritrade funciton has not been coded yet, but should be add down the line to automate pulling these tickers.

## First lets see why stocks we already own for a specific account

I only want to work with Equity investments. This is kind of confusing, but at the account level assets that can be traded are call "EQUITY". When you get quotes for each asset, the same asset can be something like "ETF".

I also use Ameritrade's portfolio planner tool to create an asset mix based off of their reccomendations. I don't want these stocks (or in my case mutual funds and ETFs) to be part of this analysis. So I'll remove them here.

In [4]:
# Specific Portfolio Account
account_portfolio_df = utils.get_account_portfolio_data(td_ameritrade.parse_portfolios_list(), masked_account_number)
equity_investments_df = utils.get_investments_by_type(account_portfolio_df, investment_type='EQUITY')

# Filter out non Equity investments
current_stocks = amc.AmeritradeRest(username, password, client_id).get_quotes(utils.get_investment_symbols(equity_investments_df)).query('assetType == "EQUITY"').index.tolist()
current_investments_df = equity_investments_df[equity_investments_df['symbol'].isin(current_stocks)]
current_investments_df

,account,shortQuantity,averagePrice,currentDayProfitLoss,currentDayProfitLossPercentage,longQuantity,settledLongQuantity,settledShortQuantity,marketValue,maintenanceRequirement,currentDayCost,previousSessionLongQuantity,assetType,cusip,symbol,description,type
6,#---9216,0.0,0.21531,-35.775002,-4.14,4500.0,4500.0,0.0,828.23,0.0,0.0,4500.0,EQUITY,Q3860H107,FGPHF,0,0


## Remove other assets

There may be some stocks that you are speculating on and do not want to be part of the analysis. Being a conservative investor, I have a percentage of my active portfolio (that is not part of the portfolio planner), that I have personally speculated on and are using for a long term play. These stocks will not be part of the portfolio optimization.

In [5]:
speculative_stocks = ['FGPHF']
final_investments_df = current_investments_df[~current_investments_df['symbol'].isin(speculative_stocks)]
final_existing_stocks = utils.get_investment_symbols(final_investments_df)
final_existing_stocks 

[]

In [6]:
symbols_of_interest = []
# Hardcoded for now
symbols_via_specific_criteria = []

In [7]:
snp_500_df = utils.get_snp500()
display(snp_500_df.head())
snp_500_symbols = snp_500_df.index.to_list()
quote_dfs = []
for i in range(0, 500, 100):
    quote_dfs.append(amc.AmeritradeRest(username, password, client_id).get_quotes(snp_500_symbols[i:i+100]))
    
snp_500_quotes_df = pd.concat(quote_dfs, axis=0)
snp_500_quotes_df.describe()

snp_500_tickers = snp_500_quotes_df.index.to_list()

,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
Symbol,,,,,,,,
MMM,3M,reports,Industrials,Industrial Conglomerates,"Saint Paul, Minnesota",1976-08-09,0000066740,1902
ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago , Illinois",1964-03-31,0000001800,1888
ABBV,AbbVie,reports,Health Care,Pharmaceuticals,"North Chicago , Illinois",2012-12-31,0001551152,2013 (1888)
ABMD,Abiomed,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,0000815094,1981
ACN,Accenture,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,0001467373,1989


In [8]:
stock_universe = set(symbols_of_interest + symbols_via_specific_criteria + final_existing_stocks + snp_500_tickers)

# Price History data

One you have a set of investments you want to work with, you will need to pull some historical data for them.

We will obtain 3 years of price histories. In the end this will provide us with 2 years of factor data since some of the factors are based on 1 year returns.

In [16]:
number_of_years = 5
price_histories = amc.AmeritradeRest(username, password, client_id).get_price_histories(stock_universe, datetime.today().strftime('%Y-%m-%d'), num_periods=number_of_years)
utils.save_price_histories(price_histories, price_histories_file_name)

Tickers:   0%|          | 0/500 [00:00<?, ?Price Histories/s]

In [11]:
price_histories.head()

,open,high,low,close,volume,ticker,date
0,88.02,88.14,84.56,85.87,10826263,STT,2018-07-20
272109,138.16,139.19,136.60,137.17,2832465,DE,2018-07-20
271354,56.08,56.14,54.83,55.02,32624511,MU,2018-07-20
270599,81.95,82.71,81.58,81.96,2439516,HLT,2018-07-20
269844,32.67,32.72,31.67,31.70,5553534,LB,2018-07-20


In [17]:
price_histories = utils.read_price_histories(price_histories_file_name)
close = utils.get_close_values(price_histories)
close.tail()

ticker,A,AAL,AAP,AAPL,ABBV,ABC,ABMD,ABT,ACN,ADBE,...,WRK,WST,WU,WY,WYNN,XEL,XLNX,XOM,XRAY,XYL
date,,,,,,,,,,,,,,,,,,,,,
2021-07-14 00:00:00+00:00,148.63,20.62,210.16,149.150,117.36,113.56,312.30,117.17,314.390,608.83,...,51.260,371.44,23.580,34.70,110.7100,68.09,132.74,59.530,62.16,121.01
2021-07-15 00:00:00+00:00,148.96,20.46,209.25,148.480,117.18,113.16,310.19,116.66,314.940,606.17,...,50.290,371.96,23.440,34.63,108.2300,68.72,129.71,58.950,61.80,120.01
2021-07-16 00:00:00+00:00,148.93,19.79,205.85,146.390,117.50,112.88,317.98,117.51,311.910,606.10,...,49.010,373.66,23.500,34.02,104.7000,69.13,128.63,57.320,61.16,119.36
2021-07-19 00:00:00+00:00,147.58,18.97,205.67,142.450,115.42,112.60,314.62,118.79,309.180,602.05,...,47.990,371.00,22.720,32.95,104.7400,68.27,129.33,55.350,60.69,117.23
2021-07-20 00:00:00+00:00,149.59,20.20,211.18,146.765,115.94,116.67,323.96,118.82,313.815,610.37,...,48.665,375.57,23.105,33.52,106.3567,68.66,130.92,56.165,62.55,121.55


In [21]:
holdings = utils.get_holdings(final_investments_df, set(close.columns.to_list()))['marketValue']
display(holdings)
utils.save_port_data(holdings.reset_index(), portfolio_file_name)

symbol
A       0.0
AAL     0.0
AAP     0.0
AAPL    0.0
ABBV    0.0
       ... 
XEL     0.0
XLNX    0.0
XOM     0.0
XRAY    0.0
XYL     0.0
Name: marketValue, Length: 500, dtype: float64

## Portfolio weights

With the portfolio stocks and the additional stocks, show how each of them contribute to the portfolio. Later, once we produce an optimized portfolio, we can generate a report on how much stock to buy/sell based of what we already have.

In [22]:
holding_weights = utils.get_portfolio_weights(holdings).fillna(0.0)
display(holding_weights)

symbol
A       0.0
AAL     0.0
AAP     0.0
AAPL    0.0
ABBV    0.0
       ... 
XEL     0.0
XLNX    0.0
XOM     0.0
XRAY    0.0
XYL     0.0
Length: 500, dtype: float64